In [39]:
from pathlib import Path
from prefect import flow, task, get_run_logger
import polars as pl
import polars.selectors as cs
import pandas as pd
import mlflow
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import xgboost as xgb

In [13]:
data_path = "/Users/zacklarsen/Documents/Projects/kaggle-wids-datathon-2020/data/"
training_v2_path = Path(data_path, "training_v2.csv")

In [15]:
training_v2_path

PosixPath('/Users/zacklarsen/Documents/Projects/kaggle-wids-datathon-2020/data/training_v2.csv')

In [3]:
%%bash
head -n 5 /Users/zacklarsen/Documents/Projects/kaggle-wids-datathon-2020/data/training_v2.csv

encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_s

In [4]:
training_v2 = pl.read_csv(training_v2_path, infer_schema_length=10000, dtypes={"age": pl.Float32}, ignore_errors=True)

In [5]:
training_v2.null_count()

encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,…,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,4228,0,0,1395,25,0,21409,112,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1662,1662


In [6]:
training_v2.null_count().transpose(include_header=True).sort(by="column_0", descending=True)

column,column_0
str,u32
"""hospital_admit…",21409
"""age""",4228
"""apache_3j_body…",1662
"""apache_2_bodys…",1662
"""ethnicity""",1395
"""icu_admit_sour…",112
"""gender""",25
"""encounter_id""",0
"""patient_id""",0


In [7]:
(
    training_v2
    .select(pl.col("age").value_counts())
)

age
struct[2]
"{39.0,526}"
"{34.0,544}"
"{54.0,1598}"
"{47.0,921}"
"{56.0,1651}"
"{86.0,1295}"
"{80.0,1702}"
"{37.0,509}"
"{46.0,908}"


In [8]:
(
    training_v2
    #.drop_nulls()
    .select(pl.col("age").value_counts(sort=True))
)

age
struct[2]
"{null,4228}"
"{67.0,2271}"
"{68.0,2173}"
"{71.0,2141}"
"{72.0,2110}"
"{66.0,2059}"
"{65.0,2051}"
"{70.0,2032}"
"{63.0,1977}"


In [31]:
X_train = pl.read_parquet(data_path + "X_train.parquet")
y_train = pl.read_parquet(data_path + "y_train.parquet")

In [34]:
X_train.glimpse()

Rows: 73370
Columns: 185
$ encounter_id                  <i64> 59954, 63518, 34401, 71581, 29019, 70076, 63091, 40553, 75009, 94808
$ patient_id                    <i64> 90449, 19384, 20558, 112066, 129440, 97970, 42207, 499, 125636, 104142
$ hospital_id                   <i64> 118, 185, 188, 62, 161, 89, 161, 186, 62, 128
$ age                           <f32> 86.0, 72.0, 36.0, 60.0, 27.0, 71.0, 35.0, 75.0, 72.0, 50.0
$ bmi                           <str> '28.16897519', '36.63508791', '27.45968419', '21.97735112', '19.960244', '21.31254263', '29.34003963', '29.1796875', '24.24339532', '22.3046875'
$ elective_surgery              <i64> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
$ ethnicity                     <str> 'Caucasian', 'Caucasian', 'African American', 'Caucasian', 'Caucasian', 'Other/Unknown', 'Hispanic', 'Caucasian', 'Caucasian', 'Caucasian'
$ gender                        <str> 'M', 'M', 'M', 'M', 'F', 'F', 'F', 'F', 'M', 'F'
$ height                        <str> '170.2', '170.1', '162.6',

In [42]:
X_train.select(cs.numeric()).glimpse()

Rows: 73370
Columns: 9
$ encounter_id          <i64> 59954, 63518, 34401, 71581, 29019, 70076, 63091, 40553, 75009, 94808
$ patient_id            <i64> 90449, 19384, 20558, 112066, 129440, 97970, 42207, 499, 125636, 104142
$ hospital_id           <i64> 118, 185, 188, 62, 161, 89, 161, 186, 62, 128
$ age                   <f32> 86.0, 72.0, 36.0, 60.0, 27.0, 71.0, 35.0, 75.0, 72.0, 50.0
$ elective_surgery      <i64> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
$ icu_id                <i64> 100, 687, 840, 481, 413, 133, 324, 498, 467, 377
$ pre_icu_los_days      <f64> 0.0, 0.088888889, 0.044444444, 0.213194444, 0.052083333, 0.0, 0.095833333, 0.399305556, 0.288888889, 0.101388889
$ readmission_status    <i64> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
$ apache_post_operative <i64> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0



In [38]:
y_train.select(pl.col("hospital_death").value_counts())

hospital_death
struct[2]
"{0,67042}"
"{1,6328}"


In [43]:
model = xgb.XGBClassifier()
model.fit(X_train.select(cs.numeric()), y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

## Function Definitions

In [56]:
@flow
def ingest_nb(path):
    data = pl.read_csv(path, infer_schema_length=10000, dtypes={"age": pl.Float32}, ignore_errors=True)
    logger = get_run_logger()
    logger.info("Data ingested")

    return data


@flow
def calc_mean_age_nb(data):
    mean_age = data.select(pl.col("age")).mean().item()
    logger = get_run_logger()
    logger.info(f"Mean age calculated as: {mean_age}")

    return mean_age


@flow
def split_nb(data, test_size=0.2, random_state=42):
    logger = get_run_logger()
    logger.info("Splitting data into train and test sets")
    X = data.drop('hospital_death')
    y = data['hospital_death']

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state)

    logger.info("Size of train set: {X_train.shape}")
    logger.info("Size of test set: {X_test.shape}")

    return X_train, X_test, y_train, y_test


@flow
def save_splits_nb(X_train, X_test, y_train, y_test):
    logger = get_run_logger()
    logger.info("Saving splits")
    X_train.write_parquet(Path(data_path, 'X_train.parquet'))
    pl.DataFrame(y_train).write_parquet(Path(data_path, 'y_train.parquet'))
    X_test.write_parquet(Path(data_path, 'X_test.parquet'))
    pl.DataFrame(y_test).write_parquet(Path(data_path, 'y_test.parquet'))

    return None


@flow
def train_model_nb(X_train, y_train):
    logger = get_run_logger()
    logger.info("Training XGBoost classifier")
    model = xgb.XGBClassifier()
    model.fit(X_train, y_train)

    return model


@flow
def predict_nb(model, test_data):
    logger = get_run_logger()
    logger.info("Predicting on test data")
    preds = model.predict(test_data)

    return preds


@flow
def evaluate_model_nb(y_test, y_pred):
    logger = get_run_logger()
    logger.info("Evaluating model performance")
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    return accuracy, precision, recall, f1, roc_auc

/Users/zacklarsen/mambaforge/envs/wids2020/lib/python3.8/site-packages/prefect/flows.py:323: UserWarning: A flow named 'ingest-nb' and defined at '/var/folders/g7/822123tx4gn6ndvw2jgh5f140000gn/T/ipykernel_19527/1641189851.py:1' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(
/Users/zacklarsen/mambaforge/envs/wids2020/lib/python3.8/site-packages/prefect/flows.py:323: UserWarning: A flow named 'calc-mean-age-nb' and defined at '/var/folders/g7/822123tx4gn6ndvw2jgh5f140000gn/T/ipykernel_19527/1641189851.py:10' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(
/Users/zacklarsen/mambaforge/envs/wids2020/lib/python3.8/site-packages/prefect/flows.py:323: UserWarning: A flow named 'split-nb' and defined at '/var/folders/g7/822123tx4gn6ndvw2jgh5f140000gn/T/ipykernel_19527/1641189851.py:1

In [ ]:
@flow
def prep_workflow(file_path):
    data = ingest_nb(file_path)
    mean_age = calc_mean_age_nb(data)
    X_train, X_test, y_train, y_test = split_nb(data, test_size=0.2, random_state=42)
    save_splits_nb(X_train, X_test, y_train, y_test)
    clf = train_model_nb(X_train.select(cs.numeric()), y_train)
    y_pred = predict_nb(clf, X_test.select(cs.numeric()))
    accuracy, precision, recall, f1, roc_auc = evaluate_model_nb(y_test, y_pred)

    result_dict = {
        'mean_age': mean_age,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': roc_auc
    }

    return result_dict

/Users/zacklarsen/mambaforge/envs/wids2020/lib/python3.8/site-packages/prefect/flows.py:323: UserWarning: A flow named 'prep-workflow' and defined at '/var/folders/g7/822123tx4gn6ndvw2jgh5f140000gn/T/ipykernel_19527/1653968534.py:1' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


In [ ]:
# Assign the flow to a variable
flow_instance = prep_workflow

In [67]:
# Run the flow
result = flow_instance(training_v2_path)

11:18:37.531 | INFO    | prefect.engine - Created flow run 'bold-bittern' for flow 'prep-workflow'

/Users/zacklarsen/mambaforge/envs/wids2020/lib/python3.8/site-packages/prefect/tasks.py:333: UserWarning: A task named 'ingest-nb' and defined at '/var/folders/g7/822123tx4gn6ndvw2jgh5f140000gn/T/ipykernel_19527/1641189851.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


11:18:37.586 | INFO    | Flow run 'bold-bittern' - Created subflow run 'adept-coucal' for flow 'ingest-nb'

11:18:37.899 | INFO    | Flow run 'adept-coucal' - Data ingested

11:18:37.939 | INFO    | Flow run 'adept-coucal' - Finished in state Completed()

...

/Users/zacklarsen/mambaforge/envs/wids2020/lib/python3.8/site-packages/prefect/tasks.py:333: UserWarning: A task named 'evaluate-model-nb' and defined at '/var/folders/g7/822123tx4gn6ndvw2jgh5f140000gn/T/ipykernel_19527/1641189851.py:66' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


11:18:39.032 | INFO    | Flow run 'bold-bittern' - Created subflow run 'pragmatic-quokka' for flow 'evaluate-model-nb'

11:18:39.056 | INFO    | Flow run 'pragmatic-quokka' - Evaluating model performance

11:18:39.090 | INFO    | Flow run 'pragmatic-quokka' - Finished in state Completed()

11:18:39.106 | INFO    | Flow run 'bold-bittern' - Finished in state Completed()

In [68]:
result

{'mean_age': 62.30951690673828,
 'accuracy': 0.9127732650057242,
 'precision': 0.32432432432432434,
 'recall': 0.007561436672967864,
 'f1': 0.014778325123152709,
 'roc_auc': 0.5030347169041612}

In [71]:
print(f"Mean Age: {result['mean_age']:.4f}")

print(f"Accuracy: {result['accuracy']:.4f}")
print(f"Precision: {result['precision']:.4f}")
print(f"Recall: {result['recall']:.4f}")
print(f"F1 score: {result['f1']:.4f}")
print(f"ROC AUC score: {result['roc_auc']:.4f}")

Mean Age: 62.3095
Accuracy: 0.9128
Precision: 0.3243
Recall: 0.0076
F1 score: 0.0148
ROC AUC score: 0.5030
